# Healthcare Claims Data Analysis Assignment

### Part 1 Data Loading and Exploration

In [1]:
# load csv files into pandas dataframes
import pandas as pd
df_header = pd.read_csv("data/df_header.csv")
df_line = pd.read_csv('data/df_line.csv')
df_code = pd.read_sql('data/df_code.csv')

# display first few rows of each dataframe
print('Header DataFrame:')
print(df_header.head(5))
print(f'\n{df_header}.shape\n')


ModuleNotFoundError: No module named 'pandas'